## Initial work


In [ ]:
%% Description              
%
% Per subject time-resolved decoding, locked to S1 onset, of
%   1. spatial frequency
%   2. orientation
%   3. phase
%
% NB: train and test sets are balanced for spatial frequency, orientation 
%     and phase of the grating
%-------------------------------------------------------------------------%

%% Init                  	
% initialize
clearvars; close all; clc;

## Parameters
Below, the parameters are given, with their default values. This cell is tagged with the tagg "parameters". Papermill would inject a cell with parameters below this one, overriding the defaults with whatever setting you want.

In [2]:
%% Parameters             	
bws        = -0.2; 	% baseline window start
bwe        = 0;   	% baseline window end
timeradius = 5;     % for temporal 'searchlight'
nfolds     = 1;	% number of folds for cross-validation
ptest      = 0.1;	% proportion of data used to assess classifier accuracy
suffix     = { 'SF_L' } ; %, 'Ori_L', 'Phase_L', ...
              % 'SF_R', 'Ori_R', 'Phase_R'} ;

% fieldtrip folder
folderFT = '/Users/casper/Downloads/fieldtrip-20190910';
folderCoSMo = '/Users/casper/Downloads/fieldtrip-20190910';

% set folders
folderIN    = fullfile(cd,'TIMELOCKED','S1');
folderTRL   = fullfile(cd,'TRIALS','REJECTVISUAL');
folderOUT   = cd;

In [3]:
% add fieldtrip path
PSR_setpaths(folderFT, folderCoSMo);

% subject
% subjects = PSR_subjects(folderIN);

## Core code
Left unchanged. Suggestion: consider throwing an error (or at least printing some message) if the run is aborted because the desired output file already exists.

In [ ]:
%% Loop over subjects     	
for isubj = 1:numel(subjects)
    %% Print message      	
    fprintf('\n%s\n',subjects{isubj});
    fileout = [folderOUT filesep subjects{isubj} '.mat'];
    if exist(fileout,'file')
        continue
    end
    
    %% Load data         	
    channels = 'MEG';
    tic
    [data]   = PSR_loadTimelock(folderIN,folderTRL,'S1',subjects{isubj},channels);
    dataTmp = data.s1lock;
    dataTmp = rmfield(dataTmp, 'cfg');
    dataTmp = ft_struct2double(dataTmp);
    toc;
    
    %% Preprocess        	
    % 1. baseline correction
    cfg = [];
    cfg.demean = 'yes';
    cfg.baselinewindow = [bws bwe];
    [dataTmp] = ft_preprocessing(cfg,dataTmp);
    
    
    
    % 2. standardize data per sensors
    i1 = find( abs(dataTmp.time{1}-bws) == min(abs(dataTmp.time{1}-bws)) );
    i2 = find( abs(dataTmp.time{1}-bwe) == min(abs(dataTmp.time{1}-bwe)) );
    bw_std = std( cell2mat( cellfun(@(x) x(:,i1:i2),dataTmp.trial,'UniformOutput',false) ), [], 2);
    dataTmp.trial = cellfun(@(x) bsxfun(@rdivide,x,bw_std), dataTmp.trial,'UniformOutput',false);
    
    
    
    % 3. select conditions
    cfg        = [];
    cfg.trials = PSR_selectCondition(dataTmp,{'fll','flh','frl','frh'});
    [dataTmp]  = ft_selectdata(cfg,dataTmp);
    
    

    % 4. make CoSMo dataset
    ds = PSR_mkCoSMoData(dataTmp);
    cosmo_check_dataset(ds);
    clear data dataTmp
    
    %% MVPA                 
    for s = 1:length(suffix)
        if  ismember( subjects{isubj}, {'19910219ANSL', '19910228FLPE'} ) && ...
                contains(suffix{s},'Phase')
            % no phase triggers for these subjects
            continue;
        end
        
        %% Partitions           
        % select data
        if s == 1
            dsTMP = cosmo_slice(ds, ds.sa.fix==0, 1);
        elseif s == 4
            dsTMP = cosmo_slice(ds, ds.sa.fix==1, 1);
        end
        
        % set targets
        target_str = suffix{s};
        target_str = target_str(1);
        switch target_str
            case 'S'
                dsTMP.sa.targets = dsTMP.sa.sf;
            case 'O'
                dsTMP.sa.targets = dsTMP.sa.ori1;
            case 'P'
                dsTMP.sa.targets = dsTMP.sa.phase;
        end
        
        % partitions
        partitions = PSR_mkCoSMoPartitions_BALANCE( dsTMP, nfolds, (1:length(dsTMP.sa.targets))', 1-ptest );
        partitions = cosmo_balance_partitions(partitions, dsTMP);
        
        train_indices = cat(2,partitions.train_indices{:});
        test_indices  = cat(2,partitions.test_indices{:});
        
        % time neihgborhood for searchlight in time
        time_nbrhood = cosmo_interval_neighborhood(dsTMP,'time','radius',timeradius);

        %% SVM classification   
        % pre-allocation
        ntime       = length(dsTMP.a.fdim.values{2});
        ntesttrials = size(test_indices,1);
        tmp_y       = int8( zeros( ntesttrials, ntime, nfolds) );
        tmp_yhat    = int8( zeros( ntesttrials, ntime, nfolds) );

        % print message
        fprintf('Classification %s\n', suffix{s} );
        counter = 0;
        tstart = tic;
        
        % loop through folds and time points
        parfor p = 1:nfolds
            
            fprintf('Fold %d/%d... ',p,nfolds);
            tic;
            
            % slice along sample attribute dimension
            ds_train = cosmo_slice(dsTMP, train_indices(:,p), 1);
            ds_test  = cosmo_slice(dsTMP, test_indices(:,p), 1);
            
            % loop over time points
            for t = 1:ntime
                % time mask: current timepoint
                timemask    = time_nbrhood.neighbors{t};
                ds_trainTMP = cosmo_slice(ds_train, timemask, 2);
                ds_testTMP  = cosmo_slice(ds_test, timemask, 2);
                
                % SVM CLASSIFICATION
                pred = cosmo_classify_svm(ds_trainTMP.samples, ds_trainTMP.sa.targets, ds_testTMP.samples);

                % add to classification arrays - where time is aligned to S1
                tmp_y(:,t,p)    = ds_testTMP.sa.targets;
                tmp_yhat(:,t,p) = pred;
            end
            
            fprintf('done! (%.2f secs)\n',toc);
        end
        
        % print message
        fprintf('\nTotal classification time: %.2f secs\n',toc(tstart));
        
        % Performance          
        % trials from all partitions
        S1.y.(suffix{s})    = double( reshape( permute(tmp_y,[1 3 2]), [], size(tmp_y,2), 1) );
        S1.yhat.(suffix{s}) = double( reshape( permute(tmp_yhat,[1 3 2]), [], size(tmp_yhat,2), 1) );
        S1.time = dsTMP.a.fdim.values{2};
        
        % accuracy
        S1.accuracy.(suffix{s}) = mean(S1.y.(suffix{s})==S1.yhat.(suffix{s}));
        
        % sensitivity locked to S1 onset
        S1.dprime.(suffix{s}) = NaN(1,ntime);
        for t = 1:ntime
            S1.dprime.(suffix{s})(t) = computeDprime( S1.y.(suffix{s})(:,t)-1, S1.yhat.(suffix{s})(:,t)-1 );
        end
        
    end
    
    %% Save data            
    save( fileout, 'S1' );
    
    % remove redundant variables
    clear ds S1
    
end